<a href="https://colab.research.google.com/github/KennethTBarrett/DS-Unit-2-Linear-Models/blob/master/KennethTBarrett_LS_DS_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [x] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [x] Engineer at least two new features. (See below for explanation & ideas.)
- [x] Fit a linear regression model with at least two features.
- [x] Get the model's coefficients and intercept.
- [x] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [x] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [x] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

# Setup Code

## Lambda's Setup Code

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

## My Setup Code

In [180]:
# Let's see what we're working with....

df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,pets_allowed
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False


# My Work

## Feature Engineering

In [0]:
# We're going to define if pets are allowed with a True / False boolean

df['pets_allowed'] = (df['cats_allowed'] == 1) & (df['dogs_allowed'] == 1)

## Split Data

In [161]:
# Splitting Data (Train Apr / May 2016, Test Jun 2016)

train = df[df['created'].str.contains('2016-04') | df['created'].str.contains('2016-05')]
test = df[df['created'].str.contains('2016-06')]

print(train.shape, test.shape)

(31844, 35) (16973, 35)


## Modeling

In [0]:
# Import 
from sklearn.linear_model import LinearRegression

In [0]:
# Instantiate

model = LinearRegression()

In [164]:
# Arrange X features matrices

features = ['bedrooms', 'bathrooms']

X_train = train[features]
X_test = test[features]
print(f'This linear regression is dependent on: {features}')
print(X_train.shape, X_test.shape)

This linear regression is dependent on: ['bedrooms', 'bathrooms']
(31844, 2) (16973, 2)


In [0]:
# Set target feature

target = 'price'

y_train = train[target]
y_test = test[target]

In [0]:
# Fit the model
model.fit(X_train, y_train)
y_pred_train = model.predict(X_train)

In [0]:
# Apply the model

y_pred_test = model.predict(X_test)

## Find Intercept and Coefficient

In [168]:
print(f'Intercept: \n {model.intercept_} \n Coefficient: \n {model.coef_}')

Intercept: 
 485.71869002322273 
 Coefficient: 
 [ 389.3248959  2072.61011639]


## Regression Metrics

In [0]:
# Imports
import numpy as np # This will be needed for sqrt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

### Train Data

In [170]:
# Regression metrics for train data

mae = mean_absolute_error(y_train, y_pred_train)
mse = mean_squared_error(y_train, y_pred_train)
rmse = np.sqrt(mse)
r2 = r2_score(y_train, y_pred_train)

print('Mean Squared Error (train):', mse)
print('Root Mean Squared Error (train):', rmse)
print('Mean Absolute Error (train):', mae)
print('R^2 (train):', r2)

Mean Squared Error (train): 1517879.6665142525
Root Mean Squared Error (train): 1232.0225917223486
Mean Absolute Error (train): 818.5310213271714
R^2 (train): 0.5111543084316605


### Test Data

In [171]:
# Regression metrics for test data

mae = mean_absolute_error(y_test, y_pred_test)
mse = mean_squared_error(y_test, y_pred_test)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred_test)

print('Mean Squared Error (test):', mse)
print('Root Mean Squared Error (test):', rmse)
print('Mean Absolute Error (test):', mae)
print('R^2 (test):', r2)

Mean Squared Error (test): 1487715.3104108903
Root Mean Squared Error (test): 1219.719357233823
Mean Absolute Error (test): 825.8987822403527
R^2 (test): 0.5213303957090346


## Try To Improve MAE

Let's start with the five steps from scratch as good practice for reinforcement.

### Modeling

In [0]:
# Step One: Imports
from sklearn.linear_model import LinearRegression

In [0]:
# Step Two: Instantiate the Class

model = LinearRegression()

In [174]:
# Step Three: Arrange X features matrices

features = ['latitude', 'longitude', 'bedrooms', 'bathrooms']

X_train = train[features]
X_test = test[features]

# Set target feature

target = 'price'

y_train = train[target]
y_test = test[target]
print(f'X_: {X_train.shape, X_test.shape}')
print(f'y_: {y_train.shape, y_test.shape}')

X_: ((31844, 4), (16973, 4))
y_: ((31844,), (16973,))


In [0]:
# Step Four: Fit the Model
model.fit(X_train, y_train)
y_pred_train = model.predict(X_train)

In [0]:
# Step Five: Apply the Model to New Data
y_pred_test = model.predict(X_test)

### Results / Regression Metrics for this Model

In [177]:
print(f'This linear regression is dependent on the following features: {features}')

This linear regression is dependent on the following features: ['latitude', 'longitude', 'bedrooms', 'bathrooms']


#### Train Data

In [178]:
# Regression metrics for train data

mae = mean_absolute_error(y_train, y_pred_train)
mse = mean_squared_error(y_train, y_pred_train)
rmse = np.sqrt(mse)
r2 = r2_score(y_train, y_pred_train)

print('Mean Squared Error (train):', mse)
print('Root Mean Squared Error (train):', rmse)
print('Mean Absolute Error (train):', mae)
print('R^2 (train):', r2)

Mean Squared Error (train): 1313709.2322857752
Root Mean Squared Error (train): 1146.1715544741874
Mean Absolute Error (train): 739.1822083746687
R^2 (train): 0.5769090842021488


#### Test Data

In [179]:
# Regression metrics for test data

mae = mean_absolute_error(y_test, y_pred_test)
mse = mean_squared_error(y_test, y_pred_test)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred_test)

print('Mean Squared Error (test):', mse)
print('Root Mean Squared Error (test):', rmse)
print('Mean Absolute Error (test):', mae)
print('R^2 (test):', r2)

Mean Squared Error (test): 1279602.324610556
Root Mean Squared Error (test): 1131.1950868928648
Mean Absolute Error (test): 744.9752658120702
R^2 (test): 0.5882903576478172
